## Hamiltonian Cycle problem solving by Recursive Best First Search algorithm with 1-Tree modified heuristic.

### 1.	Define the environment in the following block

In [10]:
#Code Block : Define appropriate classes

import math, time, tracemalloc
import heapq

class TourAgentProblem:
    """
    The Problem class to hold states related to it. It holds the graph,
    the initial state, and an object to hold its complexity analysis metrics
    """
    def __init__(self, graph, initial_state):
        self.all_nodes = graph.keys()
        self.graph = graph
        self.initial_state = initial_state

    """
    Set an object that keeps track of complexity analysis
    """
    def set_metrics(self, metrics):
        self.metrics = metrics
        self.metrics.graph_size = len(self.all_nodes)
        self.metrics.graph_edges = sum(len(neighbors) for neighbors in self.graph.values()) // 2

class Node:
    """
    The Node class to hold each node of the tree as a part of the problem.
    Each node contains its state, parent, and the path cost to it.
    """
    def __init__(self, state, parent=None, path_cost=1) -> None:
        self.state = state
        self.parent = parent
        self.path_cost = parent.path_cost + path_cost if parent else 0

    def __repr__(self) -> str:
        return str(self.state)

    def get_path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    def __lt__(self, other):
        return self

    def __le__(self, other):
        return self



In [11]:
# Holder classes for time and space complexity.

class Metrics:

    def __init__(self) -> None:
        self.nodes_expanded = 0
        self.nodes_generated = 0
        self.max_fringe_size = 0
        self.current_fringe_size = 0
        self.max_depth = 0
        self.current_depth = 0
        self.start_time = time.perf_counter()
        tracemalloc.start()

    def finalize(self):
        # End timing and memory tracking
        end_time = time.perf_counter()
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        self.execution_time_ms = (end_time - self.start_time) * 1000
        self.memory_current_kb = current / 1024
        self.memory_peak_kb = peak / 1024

    def get_algorithm(self):
        return self.algorithm

    def print_complexity_analysis(self):
        # Print detailed complexity analysis
        print(f"\n{'='*60}")
        print(f"SPACE AND TIME COMPLEXITY ANALYSIS OF {self.get_algorithm()} IMPLEMENTATION:")
        print(f"{'='*60}")

        print(f"\nGraph Statistics:")
        print(f"  • Number of nodes: {self.graph_size}")
        print(f"  • Number of edges: {self.graph_edges}")

        print(f"\nTime Complexity:")
        print(f"  • Execution time: {self.execution_time_ms:.2f} ms")
        print(f"  • Nodes expanded: {self.nodes_expanded}")
        print(f"  • Nodes generated: {self.nodes_generated}")
        print(f"  • Average branching factor: {self.nodes_generated / max(self.nodes_expanded, 1):.2f}")

        print(f"\nSpace Complexity:")
        print(f"  • Peak memory usage: {self.memory_peak_kb:.2f} KB")
        print(f"  • Current memory usage: {self.memory_current_kb:.2f} KB")
        print(f"  • Max fringe size: {self.max_fringe_size}")
        print(f"  • Max depth reached: {self.max_depth}")

        print(f"\nComplexity Estimates:")
        n = self.graph_size
        print(f"  • Theoretical worst case: O(b^d) where b=branching factor, d=depth")
        print(f"  • For Hamiltonian Cycle: O(n!) = O({n}!) without pruning")
        print(f"  • Actual nodes expanded: {self.nodes_expanded} vs theoretical max: {math.factorial(n)}")
        print(f"  • Space saved by {self.get_algorithm()}: Linear O(bd) instead of exponential O(b^d)")


class RBFSMetrics(Metrics):

    def __init__(self):
        super().__init__()
        self.algorithm = "RBFS"

In [12]:
# Define the heuristics function for the algorithm here.
def prim_mst_cost(graph, nodes):
    """
    Use Prim's algorithm to compute the minimum spanning tree cost as a heuristic
    for the remaining nodes of the graph.
    """
    if not nodes or len(nodes) < 2:
        return 0.0

    nodes_list = list(nodes)
    start = nodes_list[0]
    visited = {start}
    edges = []

    for v, w in graph[start].items():
        if v in nodes:
            heapq.heappush(edges, (w, v))

    mst_cost = 0.0
    while edges and len(visited) < len(nodes):
        w, u = heapq.heappop(edges)
        if u not in visited:
            visited.add(u)
            mst_cost += w
            for v, w2 in graph[u].items():
                if v in nodes and v not in visited:
                    heapq.heappush(edges, (w2, v))

    if len(visited) != len(nodes):
        return math.inf
    return mst_cost

#1-tree lower bound heuristic for Hamiltonian Cycle:
def get_heuristic(problem, current_node):
    visited_nodes = set([path.state for path in current_node.get_path()])
    unvisited = problem.all_nodes - visited_nodes

    if not unvisited:
        # All nodes visited, just return to start with a minimum heuristic of 0
        return 0

    # Case: only one unvisited node left
    if len(unvisited) == 1:
        last_node = next(iter(unvisited))
        cost_to_last = problem.graph[current_node.state].get(last_node, math.inf)
        cost_to_start = problem.graph[last_node].get(problem.initial_state, math.inf)
        return cost_to_last + cost_to_start

    # MST cost over unvisited nodes
    mst_cost = prim_mst_cost(problem.graph, unvisited)

    # Minimum edge from current to any unvisited node
    min_from_current = min(
        (problem.graph[current_node.state].get(v, math.inf) for v in unvisited),
        default=math.inf
    )

    # Minimum edge from any unvisited node back to start
    min_to_start = min(
        (problem.graph[v].get(problem.initial_state, math.inf) for v in unvisited),
        default=math.inf
    )

    h_val = mst_cost + min_from_current + min_to_start
    print(f"  1-tree: current={current_node.get_path()}, unvisited={unvisited}, h={h_val:.1f}")

    return h_val

In [13]:
# Code Block : Write function to design the Transition Model/Successor function.
def get_successors(problem, current_node):
    visited_nodes = set([path.state for path in current_node.get_path()]) # Don't include any node that are already visited.
    visited_nodes.remove(problem.initial_state) # Should be removed because we go back to the initial state as a part of the traversal for this problem.
    successors = [Node(child, current_node, path_cost) for (child, path_cost) in problem.graph[current_node.state].items() if (child not in visited_nodes)] # Create successors with all unvisited nodes.
    if problem.metrics:
        problem.metrics.nodes_generated += len(successors)
    return successors

In [14]:
# Code block : Write fucntion to handle goal test.
def goal_test(problem, node):
    return node.state == problem.initial_state and set([path.state for path in node.get_path()]) == problem.all_nodes


### 2.	Definition of Algorithm - Recursive Best First Search (RBFS)

In [19]:
#Code Block : Function for algorithm RBFS implementation

def rbfs(problem, node, f_limit):
    print(f"Goal Test: whether {node.get_path()} is a least cost tour.")
    if goal_test(problem, node):
        return node, node.path_cost

    successors = []
    node_f = node.path_cost + get_heuristic(problem,node)

    # Update metrics
    if problem.metrics:
        problem.metrics.max_depth = max(problem.metrics.max_depth, problem.metrics.current_depth)
        problem.metrics.max_fringe_size = max(problem.metrics.max_fringe_size, problem.metrics.current_fringe_size)

    for successor in get_successors(problem, node):
        successor_f = successor.path_cost + get_heuristic(problem, successor)

        heapq.heappush(successors, (max(node_f, successor_f), successor))
        if problem.metrics:
            problem.metrics.nodes_expanded += 1
        problem.metrics.current_fringe_size = len(successors)

    if len(successors) == 0:
        return None, float('inf')

    while successors:
        f, best = heapq.heappop(successors)
        if f > f_limit:
            return None, f
        alternative_f, _ = heapq.heappop(successors) if successors else (float('inf'), None)

        if not _:
            heapq.heappush(successors, (alternative_f, _))

        if problem.metrics:
            problem.metrics.current_depth +=1 # Increase current depth before going down the recursive path
        result, best_f = rbfs(problem, best, min(f_limit, alternative_f))

        if result:
            if problem.metrics:
                problem.metrics.current_depth -=1 # Decrease current depth because we are going a level up.
            return result, best_f

        heapq.heappush(successors, (best_f, successor))


def recursive_best_first_search(problem):
    return rbfs(problem, Node(problem.initial_state), float('inf'))

### 3. Input Graph

In [20]:
#Code Block : Graph representation

graph = {
    "A": {"B": 15, "F": 20},
    "F": {"A": 20, "E": 15, "G":20},
    "G": {"F": 20, "B": 10, "E": 10, "H": 15},
    "H": {"G": 15, "E": 25, "D": 20},
    "E": {"F": 15, "B": 15, "G": 10, "H": 25, "C": 30},
    "D": {"H": 20, "C": 5},
    "C": {"D": 5, "E": 30, "B": 10},
    "B": {"C": 10, "E": 15, "G": 10, "A": 15}
}

rbfs_problem = TourAgentProblem(graph, "A")

### 4.	Calling the search algorithms


In [23]:
# Invoke algorithm RBFS (to Print the solution, path, cost etc)
rbfs_problem.set_metrics(RBFSMetrics())
rbfs_result, path_cost = recursive_best_first_search(rbfs_problem)
print("Optimal Path found through RBFS:", rbfs_result.get_path())
print("Optimal Path Cost:", path_cost)
rbfs_problem.metrics.finalize()

Goal Test: whether [A] is a least cost tour.
  1-tree: current=[A], unvisited={'D', 'F', 'G', 'E', 'H', 'C', 'B'}, h=95.0
  1-tree: current=[A, B], unvisited={'D', 'F', 'G', 'E', 'H', 'C'}, h=95.0
  1-tree: current=[A, F], unvisited={'D', 'G', 'E', 'H', 'C', 'B'}, h=80.0
Goal Test: whether [A, F] is a least cost tour.
  1-tree: current=[A, F], unvisited={'D', 'G', 'E', 'H', 'C', 'B'}, h=80.0
  1-tree: current=[A, F, A], unvisited={'D', 'G', 'E', 'H', 'C', 'B'}, h=80.0
  1-tree: current=[A, F, E], unvisited={'D', 'G', 'H', 'C', 'B'}, h=65.0
  1-tree: current=[A, F, G], unvisited={'D', 'E', 'H', 'C', 'B'}, h=75.0
Goal Test: whether [A, F, E] is a least cost tour.
  1-tree: current=[A, F, E], unvisited={'D', 'G', 'H', 'C', 'B'}, h=65.0
  1-tree: current=[A, F, E, B], unvisited={'D', 'G', 'H', 'C'}, h=inf
  1-tree: current=[A, F, E, G], unvisited={'D', 'H', 'C', 'B'}, h=60.0
  1-tree: current=[A, F, E, H], unvisited={'D', 'G', 'C', 'B'}, h=55.0
  1-tree: current=[A, F, E, C], unvisited={'D

### 5.	Time and Space Complexity

In [22]:
#Code Block : Print the Time & Space complexity of algorithm RBFS
rbfs_problem.metrics.print_complexity_analysis()
print("="*70)


SPACE AND TIME COMPLEXITY ANALYSIS OF RBFS IMPLEMENTATION:

Graph Statistics:
  • Number of nodes: 8
  • Number of edges: 13

Time Complexity:
  • Execution time: 3.24 ms
  • Nodes expanded: 15
  • Nodes generated: 15
  • Average branching factor: 1.00

Space Complexity:
  • Peak memory usage: 23.83 KB
  • Current memory usage: 5.66 KB
  • Max fringe size: 4
  • Max depth reached: 7

Complexity Estimates:
  • Theoretical worst case: O(b^d) where b=branching factor, d=depth
  • For Hamiltonian Cycle: O(n!) = O(8!) without pruning
  • Actual nodes expanded: 15 vs theoretical max: 40320
  • Space saved by RBFS: Linear O(bd) instead of exponential O(b^d)
